# Experimental Evaluation: Goal Recognition in FOND with Temporally Extended Goals (LTLf and PLTL)

In [551]:
BASE_DIR = '../fond-recognition-benchmarks'

In [552]:
import os
import sys
import pandas as pd
import glob
import json
import numpy as np
import itertools

In [553]:
def compute_posterior(index, prob, priors):
    num = prob[index] * priors[index]
    denom = np.sum(prob*priors)
    return num / denom

In [554]:
RESULTS_DIR = os.path.join(BASE_DIR, 'results')

In [555]:
json_files = glob.glob(os.path.join(RESULTS_DIR, '*.json'))

In [556]:
print("Results found:", len(json_files))

Results found: 19


# Results Collection

In [557]:
print(json_files[0])
print(os.path.basename(json_files[0]))
filename, ext = os.path.basename(json_files[0]).split('.')

../fond-recognition-benchmarks/results/triangle-tireworld_p01_hyp-1_10_2.json
triangle-tireworld_p01_hyp-1_10_2.json


In [558]:
tokens = filename.split('_')

In [559]:
dataset = {
    'domain' : [],\
    'problem' : [],\
    'observability' : [],\
    'goals' : [],\
    'judge_point' : [],\
    'obs_len' : [],\
    'true_goal': [],\
    'time': [],\
    'posterior': []}

sum_obs_dict = dict()
problems_obs_dict = dict()

for pathname in json_files:
    filename, ext = os.path.basename(pathname).split('.')
    tokens = filename.split('_')
    approach = tokens[0]
    model_type = tokens[-1]
    data = {}
    with open(pathname) as instream:
        buffer = instream.read()
        data = json.loads(buffer)
    #print(approach, model_type, data['domain'], data['problem'], data['observability'])
    num_goals = len(data["G"])
    true_goal = data["G"].index(data['G*'])
    likelihoods = data['P(Obs | G)']
    
    if data['observability'] in sum_obs_dict:
        problems_obs_dict[data['observability']] += 1
        sum_obs_dict[data['observability']] += len(data['Obs'])
    else:
        problems_obs_dict[data['observability']] = 1
        sum_obs_dict[data['observability']] = len(data['Obs'])
    
    for k, prob_O_G in enumerate(likelihoods):
        post_probs = [compute_posterior(j, prob_O_G, np.ones(num_goals)/num_goals) \
                     for j in range(len(data['G']))]
        dataset['domain'] += [data['domain']]
        dataset['problem'] += [data['problem']]
        dataset['goals'] += [num_goals]
        dataset['judge_point'] += [k]
        dataset['obs_len'] += [len(data['Obs'])]
        dataset['true_goal'] += [true_goal]
        dataset['observability'] += [data['observability']]
        dataset['time'] += [data['time']]
        dataset['posterior'] += [np.array(post_probs)]

avg_obs = 0.0
sum_obs = 0.0
avg_obs_observability = dict()
for k in sum_obs_dict.keys():
    avg = sum_obs_dict[k] / problems_obs_dict[k]
    avg_obs_observability[k] = avg
    sum_obs += avg
    
avg_obs = sum_obs / len(sum_obs_dict)

In [560]:
dataset = pd.DataFrame(dataset)

In [561]:
def compute_accuracy(dataset, top_k):
    tp_count = 0
    for index, row in dataset.iterrows():
        top_k_goals = np.argpartition(row['posterior'], -top_k)[-top_k:]
        if row['true_goal'] in top_k_goals:
            tp_count += 1
    return tp_count / len(dataset)

In [572]:
def compute_recognition_time(dataset):
    sum_time = 0
    for index, row in dataset.iterrows():
        sum_time += row['time']

    return (sum_time / len(dataset))

In [573]:
def compute_error_rates(dataset):
    tpr = 0.0
    fpr = 0.0
    fnr = 0.0

    for index, row in dataset.iterrows():
        tp_count = 0
        tn_count = 0
        fp_count = 0
        fn_count = 0

        top_goal = np.argmax(row['posterior'])
        max_goals = [ k for k, p in enumerate(row['posterior']) if p == row['posterior'][top_goal]]
        
        if row['true_goal'] in max_goals:
            tp_count = 1
        fn_count = 1 - tp_count
        fp_count = len(max_goals) - tp_count
        tn_count = row['goals'] - fp_count
    
        tpr += tp_count/(tp_count + fp_count)
        fpr += fp_count/(fp_count + tn_count)
        fnr += fn_count/(fn_count + tp_count)
    return tpr/len(dataset), fpr/len(dataset), fnr/len(dataset)

## Online Goal Recognition

In [574]:
approaches = ['approach']
observabilities = ['10', '25', '30', '50', '70', '75', '100']

summary = {'approach': [], 'observability': [], 'top1': [], 'top2': [], 'top3': [], 'tpr' : [], 'fpr':[], 'fnr': []}

for approach, observability in itertools.product(approaches, observabilities):
    df = dataset.loc[(dataset['observability'] == observability)]
    if len(df) == 0: 
        continue
    top_1_acc = compute_accuracy(df, 1)
    top_2_acc = compute_accuracy(df, 2)
    top_3_acc = compute_accuracy(df, 3)
    tpr, fpr, fnr = compute_error_rates(df)
    summary['approach'] += [approach]
    summary['observability'] += [observability]
    summary['top1'] += [top_1_acc]
    summary['top2'] += [top_2_acc]
    summary['top3'] += [top_3_acc]
    summary['tpr'] += [tpr]
    summary['fpr'] += [fpr]
    summary['fnr'] += [fnr]

In [575]:
summary = pd.DataFrame(summary)

In [576]:
summary[['observability', 'top1', 'top2', 'top3', 'tpr', 'fpr', 'fnr']]

,observability,top1,top2,top3,tpr,fpr,fnr
0,10,0.666667,1.0,1.0,0.833333,0.111111,0.0
1,25,0.833333,1.0,1.0,0.916667,0.055556,0.0
2,30,0.666667,1.0,1.0,0.833333,0.111111,0.0
3,50,0.750000,1.0,1.0,0.875000,0.083333,0.0
4,70,0.600000,1.0,1.0,0.800000,0.133333,0.0
5,75,0.722222,1.0,1.0,0.861111,0.092593,0.0
6,100,0.714286,1.0,1.0,0.857143,0.095238,0.0


## Offline Goal Recognition

In [577]:
offline = {'observability': [], 'avg_obs': [], 'time': [], 'top1': [], 'top2': [], 'top3': [], 'tpr' : [], 'fpr':[], 'fnr': []}

for approach, observability in itertools.product(approaches, observabilities):
    df = dataset.loc[(dataset['observability'] == observability)\
                    & (dataset['judge_point'] == dataset['obs_len'] - 1)]
    if len(df) == 0: continue
    top_1_acc = compute_accuracy(df, 1)
    top_2_acc = compute_accuracy(df, 2)
    top_3_acc = compute_accuracy(df, 3)
    tpr, fpr, fnr = compute_error_rates(df)
    avg_time = compute_recognition_time(df)
    offline['observability'] += [observability]
    offline['avg_obs'] += [avg_obs_observability[observability]]
    offline['time'] += [avg_time]
    offline['top1'] += [top_1_acc]
    offline['top2'] += [top_2_acc]
    offline['top3'] += [top_3_acc]
    offline['tpr'] += [tpr]
    offline['fpr'] += [fpr]
    offline['fnr'] += [fnr]

In [578]:
offline = pd.DataFrame(offline)
offline[['observability', 'avg_obs', 'time', 'top1', 'top2', 'top3', 'tpr', 'fpr', 'fnr']]

,observability,avg_obs,time,top1,top2,top3,tpr,fpr,fnr
0,10,1.0,3.735506,0.666667,1.0,1.0,0.833333,0.111111,0.0
1,25,2.0,3.364115,1.000000,1.0,1.0,1.000000,0.000000,0.0
2,30,3.0,3.564134,1.000000,1.0,1.0,1.000000,0.000000,0.0
3,50,4.0,3.306316,1.000000,1.0,1.0,1.000000,0.000000,0.0
4,70,5.0,3.232762,1.000000,1.0,1.0,1.000000,0.000000,0.0
5,75,6.0,3.491696,1.000000,1.0,1.0,1.000000,0.000000,0.0
6,100,7.0,3.796831,1.000000,1.0,1.0,1.000000,0.000000,0.0


In [569]:
print('Average observations: %s' % avg_obs)

Average observations: 4.0


### First Observation

In [570]:
first_obs = {'observability': [], 'top1': [], 'top2': [], 'top3': [], 'tpr' : [], 'fpr':[], 'fnr': []}

for approach, observability in itertools.product(approaches, observabilities):
    df = dataset.loc[(dataset['observability'] == observability)\
                    & (dataset['judge_point'] == 0)]
    if len(df) == 0: continue
    #print(len(df))
    top_1_acc = compute_accuracy(df, 1)
    top_2_acc = compute_accuracy(df, 2)
    top_3_acc = compute_accuracy(df, 3)
    tpr, fpr, fnr = compute_error_rates(df)
    first_obs['observability'] += [observability]
    first_obs['top1'] += [top_1_acc]
    first_obs['top2'] += [top_2_acc]
    first_obs['top3'] += [top_3_acc]
    first_obs['tpr'] += [tpr]
    first_obs['fpr'] += [fpr]
    first_obs['fnr'] += [fnr]

In [571]:
first_obs = pd.DataFrame(first_obs)
first_obs[['observability', 'top1', 'top2', 'top3', 'tpr', 'fpr', 'fnr']]

,observability,top1,top2,top3,tpr,fpr,fnr
0,10,0.666667,1.0,1.0,0.833333,0.111111,0.0
1,25,0.666667,1.0,1.0,0.833333,0.111111,0.0
2,30,0.000000,1.0,1.0,0.500000,0.333333,0.0
3,50,0.000000,1.0,1.0,0.500000,0.333333,0.0
4,70,0.000000,1.0,1.0,0.500000,0.333333,0.0
5,75,0.000000,1.0,1.0,0.500000,0.333333,0.0
6,100,0.000000,1.0,1.0,0.500000,0.333333,0.0
